### required imports

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor
from datetime import datetime
from sklearn.metrics import r2_score
from catboost import Pool
import optuna
from sklearn.metrics import mean_squared_error

### importing the clean pre processed data (preparing it a little bit)

In [17]:
data=pd.read_csv('./CleanData.csv')
data = data.drop(columns=['Unnamed: 0','Year1','Year2','Year3','day','date enregistrement','Classification_Equipement'])
data['Date Fin'] = pd.to_datetime(data['Date Fin']).apply(lambda x: x.toordinal())
data['Date début'] = pd.to_datetime(data['Date début']).apply(lambda x: x.toordinal())
data['date_enreg'] = pd.to_datetime(data['date_enreg']).apply(lambda x: x.toordinal())
data.head(15)

,Ordre de travail,Description,Equipment,Nom Equipement,Noeud Parent,Nom_parent,Date début,Date Fin,Type OT,day_diff,date_enreg
0,OT000001,Pas de Descriprion,GCULZ201,Production Beurre creme fraiche,GCULZ2,Zone 2,737119,737119,correctif planifié,16,737103
1,OT000002,Pas de Descriprion,GCULZ103,Production UHT,GCULZ1,Zone 1,737091,737091,correctif planifié,16,737075
2,OT000003,Pas de Descriprion,ULPRPH0104,TBA/8,ULPRPH01,UHT A,737119,737119,correctif planifié,16,737103
3,OT000011,Pas de Descriprion,ULPRPH0104,TBA/8,ULPRPH01,UHT A,737119,737119,correctif planifié,16,737103
4,OT000012,Préparation les nouveaux têtes scellage,ULPRPS0204,REMPLISSEUSE-CAPSULEUSE,ULPRPS02,BOUTEILLE,737091,737091,correctif planifié,-1,737091
5,OT000021,Pas de Descriprion,ULPRPT0101,STATION DOSAGE SOUDE ET ACIDE CONCENTRE,ULPRPT01,NEP,737091,737119,correctif planifié,16,737075
6,OT000023,Pas de Descriprion,PFC105,Posimat A,PFC1,conditionnement 1,737091,737091,correctif non planifié,-1,737091
7,OT000031,Pas de Descriprion,ULUTEG0106,circuit eau glacé,ULUTEG01,PRODUCTION EAU GLACCE,737091,737119,correctif planifié,16,737075
8,OT000064,Pas de Descriprion,PFC109,Sleever B (10000853-417),PFC1,conditionnement 1,737119,737119,correctif planifié,-1,737119
9,OT003799,réparation éclairage au niveau atelier UHT,ULPRPH0608,ECLAIRAGE,ULPRPH06,DIVERS UHT,737119,737119,correctif planifié,-1,737119


In [18]:
X= data.drop(columns=['Date début'])
Y= data['Date début']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)   

### creating and training the model

In [108]:
cat_features = [
    'Ordre de travail',
    'Description',
    'Equipment',
    'Nom Equipement',
    'Noeud Parent',
    'Nom_parent',
    'Type OT'
]

# Split your data into training and validation sets

# Initialize the model with hyperparameters
cb_regressor = CatBoostRegressor(iterations=4899,
                                 learning_rate=0.1,
                                 depth=10,
                                 loss_function='RMSE',
                                 verbose=100)

# Fit the model with early stopping
cb_regressor.fit(X_train, Y_train, 
                 cat_features=cat_features, 
                 eval_set=(X_test, Y_test),   # Validation set to monitor the performance
                 early_stopping_rounds=100, # Stop after 100 rounds without improvement
                 verbose=100)

0:	learn: 315.4080141	test: 317.1455331	best: 317.1455331 (0)	total: 100ms	remaining: 8m 10s
100:	learn: 7.8263203	test: 9.4613859	best: 9.4613859 (100)	total: 7.76s	remaining: 6m 8s
200:	learn: 4.7959785	test: 6.5641721	best: 6.5641721 (200)	total: 18.8s	remaining: 7m 19s
300:	learn: 3.4104898	test: 5.4211199	best: 5.4211199 (300)	total: 42.3s	remaining: 10m 45s
400:	learn: 2.7196280	test: 4.9614745	best: 4.9614745 (400)	total: 1m 3s	remaining: 11m 46s
500:	learn: 2.2231038	test: 4.7146129	best: 4.7134372 (499)	total: 1m 28s	remaining: 12m 58s
600:	learn: 1.8677043	test: 4.5511512	best: 4.5511512 (600)	total: 1m 50s	remaining: 13m 8s
700:	learn: 1.5944252	test: 4.4472948	best: 4.4472948 (700)	total: 2m 12s	remaining: 13m 12s
800:	learn: 1.3953620	test: 4.3686741	best: 4.3686741 (800)	total: 2m 34s	remaining: 13m 11s
900:	learn: 1.2430710	test: 4.3236857	best: 4.3236857 (900)	total: 2m 42s	remaining: 12m 1s
1000:	learn: 1.1186209	test: 4.2855691	best: 4.2855691 (1000)	total: 2m 52s	rem

### Evaluating the model

In [ ]:
y_pred = cb_regressor.predict(X_test)
samDay = np.mean(np.abs(np.array(Y_test) - np.array(y_pred)) <= 3 )
print(f"Percentage of predictions that were on the same day of the actual date: {samDay * 100:.2f}%")

: 

In [110]:
from sklearn.metrics import mean_absolute_error


y_train_pred = cb_regressor.predict(X_train)
y_test_pred = cb_regressor.predict(X_test)

print("Train MAE:", mean_absolute_error(Y_train, y_train_pred))
print("Test MAE:", mean_absolute_error(Y_test, y_test_pred))
print("Train R²:", r2_score(Y_train, y_train_pred))
print("Test R²:", r2_score(Y_test, y_test_pred))

Train MAE: 0.9735768825172849
Test MAE: 1.4865130483582294
Train R²: 0.9999417197063415
Test R²: 0.9998619757086942


In [111]:
y_pred = cb_regressor.predict(X_test)

y_pred_dates = [datetime.fromordinal(int(pred)) for pred in y_pred]
y_test_dates = [datetime.fromordinal(int(actual)) for actual in Y_test]


In [ ]:

#optional =  automatically guessing what the best hyperparameters are for the CatBoost model using Optuna
# Define the objective function for Optuna optimization



#def objective(trial):
#    # Hyperparameters to tune
#    learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.3)
#    depth = trial.suggest_int('depth', 6, 12)  # Depth of trees
#    l2_leaf_reg = trial.suggest_loguniform('l2_leaf_reg', 1, 10)  # L2 regularization
#    bagging_temperature = trial.suggest_uniform('bagging_temperature', 0, 1)  # Bagging temperature
#    random_strength = trial.suggest_int('random_strength', 1, 10)  # Random strength
#    
#    # Initialize CatBoostRegressor with suggested hyperparameters
#    model = CatBoostRegressor(
#        learning_rate=learning_rate,
#        depth=depth,
#        iterations=4899,
#        loss_function='RMSE',
#        l2_leaf_reg=l2_leaf_reg,
#        bagging_temperature=bagging_temperature,
#        random_strength=random_strength,
#        early_stopping_rounds=100,  # Early stopping
#        cat_features=cat_features,
#        verbose=0  # Suppress logs during trials
#    )
#    
#    # Fit the model with the training data
#    model.fit(X_train, Y_train, eval_set=(X_test, Y_test))
#    
#    # Predict on the validation set
#    y_pred = model.predict(X_test)
#    
#    # Calculate RMSE (root mean squared error)
#    rmse = np.sqrt(mean_squared_error(Y_test, y_pred))
#    
#    return rmse
#
## Run the optimization to find the best hyperparameters
#study = optuna.create_study(direction='minimize')  # Minimize RMSE
#study.optimize(objective, n_trials=20)  # Number of trials to run
#
## Get the best hyperparameters
#best_params = study.best_params
#print("Best Hyperparameters:", best_params)
#
## Train the final model with the best parameters
#best_model = CatBoostRegressor(
#    **best_params,
#    early_stopping_rounds=100,
#    cat_features=cat_features,
#    verbose=100
#)
#best_model.fit(X_train, Y_train, eval_set=(X_test, Y_test))
#
## Evaluate the final model
#final_predictions = best_model.predict(X_test)
#final_rmse = np.sqrt(mean_squared_error(Y_test, final_predictions))
#print(f"Final RMSE: {final_rmse:.2f}")
